<font size=4>**(2 late days)<br>name: Yonatan Juarez<br>github: yonatan-juarez-5**<br>

<font size=3>**Classification of time series<br>
Feature creation/extraction**

In [19]:
from tqdm import tqdm
import os
import pandas as pd
from scipy.stats import bootstrap
# import bootstrapped.bootstrap as bs
# from bootstrapped.stats_functions import std
from tabulate import tabulate
import numpy as np

In [20]:
# for i in tqdm(range(1,1000000)):
#     print(i, end=" ")

<font size=4>**1a) Download the AReM data:<br>
1b) Keep datasets 1/2 in bending1/2 and datasets 1/2/3 in other folders as test data<br>
The rest of the data sets will be train data**

In [21]:
filepath = "../data/AReM/"
folder_list = os.listdir(filepath)
# print(file_list)
folder_list.remove(".DS_Store")
folder_list.remove("bendingType.pdf")
folder_list.remove("sensorsPlacement.pdf")
print(folder_list)
file_dict = {}
# print(filepath + folder_list[0])

['bending1', 'walking', 'bending2', 'standing', 'sitting', 'lying', 'cycling']


In [22]:
test_data = {}
train_data = {}
for folder in folder_list:
    folderpath = filepath + folder
    temp_list = os.listdir(folderpath)
    # temp_list.sort()
    for file in temp_list:
        # print(file)
        path = os.path.join(folderpath, file)
        # print(path)
        file_dict[path] = pd.read_csv(path, skiprows=4, usecols=[1,2,3,4,5,6])

        if folder == "bending1" or folder == "bending2":
            if file == "dataset1.csv" or file == "dataset2.csv":
                test_data[path] = file_dict[path]
            else:
                train_data[path] = file_dict[path]
        else:
            if file == "dataset1.csv" or file == "dataset2.csv" or file == "dataset3.csv":
                test_data[path] = file_dict[path]
            else:
                train_data[path] = file_dict[path]


# print(train_data)
print(f"Data split\nTraining data set: {len(train_data)}\nTest data set: {len(test_data)}")


Data split
Training data set: 69
Test data set: 19


In [23]:
# print("\tTraining data")
# train_data

In [24]:
# print("\tTest data")
# test_data

<font size=5>**(1ci) Research what types of T-D features are usually used in time series classification**
<font size=4>
- Some common time-domain features used are:
    - Mean:
        - Represents the average value of the time series.
    - Median:
        - Represents the middle value in the sorted time series.
    - Standard Deviation:
        - Measures the spread or dispersion of values around the mean.
    - Variance:
        - Measures the average squared deviation from the mean.
    - Skewness:
        - Describes the asymmetry of the distribution of values.
    - Kurtosis:
        - Measures the "tailedness" or peakedness of the distribution.
    - Minimum and Maximum:
        - The minimum and maximum values in the time series.
    - Range:
        - The difference between the maximum and minimum values.
    - Quantiles:
        - Values that divide the data into intervals of equal probability (e.g., quartiles, quintiles).
    - Hurst Exponent:
        - Describes the long-term memory of the time series.

<font size=5>**1cii) Feature extraction:<br>
classification of time series usually needs extracting features.<br>
In this problem we focus on time-deoman features**

In [25]:
col_headers = ['min', 'max', 'mean', 'median', 'std', '1Q', '3Q']
stats_headers = ['min', 'max', 'mean','50%', 'std', '25%', '75%']
columns=[]
for i in range(6):
    for col in col_headers:
        columns.append(f"{col}-{i+1}")

extracted_data = []
for file, data in file_dict.items():
    stats_df = data.describe() # stats
    cols = file_dict[file].columns # column list
    row_data = []
    for col in cols:
        for header in stats_headers:
            row_data.append(stats_df[col][header])
    extracted_data.append(row_data)

extracted_df = pd.DataFrame(extracted_data, columns=columns)
extracted_df
# print(f"\tExtracted time-domain features:\n{tabulate(extracted_df, headers='keys', tablefmt='pretty')}") 

,min-1,max-1,mean-1,median-1,std-1,1Q-1,3Q-1,min-2,max-2,mean-2,...,std-5,1Q-5,3Q-5,min-6,max-6,mean-6,median-6,std-6,1Q-6,3Q-6
0,36.25,48.00,43.969125,44.50,1.618364,43.3100,44.67,0.0,1.50,0.413125,...,3.318301,20.5000,23.7500,0.0,2.96,0.555312,0.490,0.487826,0.0000,0.8300
1,37.00,48.00,43.454958,43.25,1.386098,42.5000,45.00,0.0,1.58,0.378083,...,2.488862,22.2500,24.0000,0.0,5.26,0.679646,0.500,0.622534,0.4300,0.8700
2,33.00,47.75,42.179812,43.50,3.670666,39.1500,45.00,0.0,3.00,0.696042,...,3.849448,30.4575,36.3300,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.0000
3,33.00,45.75,41.678063,41.75,2.243490,41.3300,42.75,0.0,2.83,0.535979,...,2.411026,28.4575,31.2500,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.5000
4,37.25,45.00,40.624792,40.50,1.476967,39.2500,42.00,0.0,1.30,0.358604,...,2.188449,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,18.50,44.25,35.752354,36.00,4.614802,33.0000,39.33,0.0,12.60,3.328104,...,3.120057,14.0000,18.0625,0.0,9.39,3.069667,2.770,1.748326,1.7975,4.0600
84,24.25,45.00,37.177042,36.25,3.581301,34.5000,40.25,0.0,8.58,2.374208,...,2.890347,17.9500,21.7500,0.0,9.34,2.921729,2.500,1.852600,1.5000,3.9000
85,23.33,43.50,36.244083,36.75,3.822016,33.4575,39.25,0.0,9.71,2.736021,...,3.617702,15.7500,21.0000,0.0,11.15,3.530500,3.110,1.963685,2.1700,4.6175
86,26.25,44.25,36.957458,36.29,3.434863,34.5000,40.25,0.0,8.64,2.420083,...,2.851673,14.0000,18.2500,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.0300


<font size=5>**1ciii) Feature extraction:<br>
Estimate the std of each of the time-domain features youn extracted from the data<br>
Then use Python's bootstrapped to build 90% bootstrap condience interval for the std feature<br>**


In [26]:
std_list = []
stds = []

# for col in extracted_df_cols:
for col in list(extracted_df.columns):
    #append [feature, std] to std_list
    stds.append(extracted_df[col].std())
    std_list.append([col, stds[-1]])

headers = ['Features', 'STD']
# convert std_list to 2-d dataframe
std_df = pd.DataFrame(std_list, columns = headers)
# print(tabulate(std_df, headers='keys', tablefmt='pretty'))

std_df

,Features,STD
0,min-1,9.569975
1,max-1,4.394362
2,mean-1,5.335718
3,median-1,5.440054
4,std-1,1.772153
5,1Q-1,6.153590
6,3Q-1,5.138925
7,min-2,0.000000
8,max-2,5.062729
9,mean-2,1.574164


In [27]:
bs_data = []

idx = 0
for feature in extracted_df.columns:
    bs_result= bootstrap((extracted_df[feature], ), np.std, confidence_level = 0.9, 
                              random_state = 1, method = 'percentile')
    bs_data.append([feature, stds[idx], bs_result.confidence_interval.low, 
                                bs_result.confidence_interval.high])
    idx += 1

bs_df = pd.DataFrame(bs_data, columns = ['Feature', 'STD', 'Lower Bound', 'Upper Bound'])
bs_df


,Feature,STD,Lower Bound,Upper Bound
0,min-1,9.569975,8.217348,10.704231
1,max-1,4.394362,3.306005,5.269063
2,mean-1,5.335718,4.675214,5.837115
3,median-1,5.440054,4.760294,5.964649
4,std-1,1.772153,1.562763,1.935570
5,1Q-1,6.153590,5.542312,6.603819
6,3Q-1,5.138925,4.308947,5.806972
7,min-2,0.000000,0.000000,0.000000
8,max-2,5.062729,4.595134,5.377385
9,mean-2,1.574164,1.393100,1.698068


<font size=5>**(1civ) Use judgement to select the 3 most important T-D features**
- Median: Median can help remove outliers, which will have an impact on the mean,<br>
where outliers can help shift the mean away from the median. So the median gives<br>a rough idea of where the center of the data is
- 1st Q: 1st quartile can be used to determine the median of the lower 50th percentile, and determine<br>
any outliers 1.5*IQR below the 1st quartile.
- 3rd Q: 3rd quartile can be used to determine the median of the upper 50th percentile, and determine<br>
any outliers 1.5*IQR above the 3rd quartile.

<font size=5>**(2) ISLR 3.7.4**<br>
(a)Suppose that the true relationship between X and Y is linear, i.e. Y = β0 + β1X + ε. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.
<font size=4>
- When we add more features to a model, it will increase the flexibility and will be able to<br>
fit the training data more closely. Since a cubic rgeression model has more features than a<br>linear regression model, it will better fit the training data. However, you run the risk of overfitting your training data.<br>
This is because as flexibility goes up, so does the risk of overfitting. In conclusion, I would expect<br>
the cubic model to have a lesser RSS than the linear model.

<font size =5>**(b)Answer (a) using test rather than training RSS.**
<font size=4>
- As a result of a higher risk of overfitting with the cubic regression model, I expect the test RSS<br>
to be higher for cubic than than the linear model.

<font size = 5>**(c)Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.**
<font size=4> 
- Since the relationship betwen X and Y is not linear, a linear model would underfit the data. <br>
I would expect the training RSS for cubic regression to be less than the linear model.

<font size=5>**(d)Answer (c) using test rather than training RSS.**
<font size=4>
- Since it is not specified how far the data is from being linear, I am unable to determine whether the <br>
test RSS would be less for linear or cubic. The relationship between quadratic,, cubic, or higher order<br>
will have an affect on this